### GENUINE NOTEBOOK

In [1]:
#my imports
import numpy as np
import unicodedata
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import string
from nltk import pos_tag
import pandas as pd
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('display.max_colwidth', -1)
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score
pd.set_option("display.max_rows", None, "display.max_columns", None)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import random

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  del sys.path[0]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
#Real Testing data
low_pizza_full = pd.read_csv("https://raw.githubusercontent.com/success81/Synthetic_NLP_Data_Generation_Paper/main/pizza_low_400%20-%20pizza_low_800.csv")
high_pizza_full = pd.read_csv("https://raw.githubusercontent.com/success81/Synthetic_NLP_Data_Generation_Paper/main/pizza_high_400%20-%20pizza_high_400.csv")
low_test = low_pizza_full[300:400]
high_test = high_pizza_full[300:400]
new_test = pd.concat([low_test,high_test], axis=0)
new_x_train, new_x_test, new_y_train, new_y_test = train_test_split(new_test['text'], new_test['rating'], random_state = 0, test_size = .99)

In [3]:
#Dataframes
high_gpt = pd.read_csv("https://raw.githubusercontent.com/success81/Synthetic_NLP_Data_Generation_Paper/main/positive_gpt.csv")
low_gpt = pd.read_csv("https://raw.githubusercontent.com/success81/Synthetic_NLP_Data_Generation_Paper/93eb3de1fe859e1e74d22d14634266bcc8a896f2/negative_gpt.csv")
add_gpt_high = pd.read_csv("https://raw.githubusercontent.com/success81/Synthetic_NLP_Data_Generation_Paper/main/mega_high_df")
add_gpt_low = pd.read_csv("https://raw.githubusercontent.com/success81/Synthetic_NLP_Data_Generation_Paper/main/mega_low_df")
high_no_gpt = pd.read_csv("https://raw.githubusercontent.com/success81/Synthetic_NLP_Data_Generation_Paper/main/high_pizza_no_gpt.csv")
low_no_gpt = pd.read_csv("https://raw.githubusercontent.com/success81/Synthetic_NLP_Data_Generation_Paper/main/low_pizza_no_gpt.csv")

In [4]:
#Key Datasets
genuine_final = pd.concat([high_no_gpt,low_no_gpt], axis = 0)
gpt_only_final = pd.read_csv('https://raw.githubusercontent.com/success81/Synthetic_NLP_Data_Generation_Paper/93eb3de1fe859e1e74d22d14634266bcc8a896f2/total_mega.csv')
best_genuine_and_synthetic = pd.concat([genuine_final,gpt_only_final], axis=0, ignore_index=True)

In [5]:
#clear nulls
genuine_final = genuine_final.dropna()
gpt_only_final = gpt_only_final.dropna()
best_genuine_and_synthetic = best_genuine_and_synthetic.dropna()

In [6]:
#Drop columns
genuine_final.drop(["Unnamed: 0"], axis = 1, inplace = True)
best_genuine_and_synthetic.drop(["Unnamed: 0"], axis = 1, inplace = True)
gpt_only_final.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [7]:
best_genuine_and_synthetic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11380 entries, 0 to 11388
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   stars   11380 non-null  int64 
 1   rating  11380 non-null  object
 2   text    11380 non-null  object
dtypes: int64(1), object(2)
memory usage: 355.6+ KB


In [8]:
gpt_only_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10930 entries, 1 to 10938
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   stars   10930 non-null  int64 
 1   rating  10930 non-null  object
 2   text    10930 non-null  object
dtypes: int64(1), object(2)
memory usage: 341.6+ KB


In [9]:
genuine_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450 entries, 0 to 224
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   stars   450 non-null    int64 
 1   rating  450 non-null    object
 2   text    450 non-null    object
dtypes: int64(1), object(2)
memory usage: 14.1+ KB


In [10]:
#TEST SET LENGTH
len(new_y_test)

198

### **Bayes Model Building**

In [36]:
#This is the Training of the Genuine Naive Bayes Model For Car Reviews
X_train, X_test, y_train, y_test = train_test_split(genuine_final['text'], genuine_final['rating'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [37]:
#New Predict
naive_bayes_predict = clf.predict(count_vect.transform(new_x_test))

In [38]:
#Precision Score NO GPT
precision_score(new_y_test, naive_bayes_predict, average="weighted")

0.8281092012133467

In [39]:
#Recall Score
recall_score(new_y_test, naive_bayes_predict, average="weighted")

0.797979797979798

In [40]:
#accuracy_score
accuracy_score(new_y_test, naive_bayes_predict)

0.797979797979798

In [41]:
#F1
f1_score(new_y_test, naive_bayes_predict, average="weighted")

0.793233082706767

In [43]:
#Confusion Matrix
cm = confusion_matrix(new_y_test, naive_bayes_predict)
print (cm)

[[94  5]
 [35 64]]


### **Random Forest**

In [18]:
gpt_r_clf = RandomForestClassifier(max_depth=6, random_state=0)
gpt_r_clf.fit(X_train_tfidf, y_train)
gpt_r_predict = gpt_r_clf.predict(count_vect.transform(new_x_test))

In [19]:
accuracy_score(new_y_test, gpt_r_predict)

0.7121212121212122

In [20]:
precision_score(new_y_test, gpt_r_predict, average="weighted")

0.7424207089552239

In [21]:
recall_score(new_y_test, gpt_r_predict, average="weighted")

0.7121212121212122

In [22]:
#F1
f1_score(new_y_test, gpt_r_predict, average="weighted")

0.7028357776666051

In [23]:
#Confusion Matrix
cm = confusion_matrix(new_y_test, gpt_r_predict)
print (cm)

[[53 46]
 [11 88]]


### **Extra trees**

In [24]:
#ExtraTreesClassifier
extra_clf = ExtraTreesClassifier(bootstrap = True, max_leaf_nodes = 100, n_estimators = 350)
extra_clf.fit(X_train_tfidf, y_train)
extra_clf_predict = extra_clf.predict(count_vect.transform(new_x_test))

In [25]:
accuracy_score(new_y_test, extra_clf_predict)

0.7929292929292929

In [26]:

precision_score(new_y_test, extra_clf_predict, average="weighted")

0.8067307692307693

In [27]:
recall_score(new_y_test, extra_clf_predict, average="weighted")

0.7929292929292929

In [28]:
f1_score(new_y_test, extra_clf_predict, average="weighted")

0.790573485024379

In [29]:
cm = confusion_matrix(new_y_test, gpt_r_predict)
print (cm)

[[53 46]
 [11 88]]


### **Gradient Boosting**


In [30]:
g_clf = GradientBoostingClassifier(random_state=0, learning_rate=1, max_depth=15, min_samples_leaf = 100)
g_clf.fit(X_train_tfidf, y_train)
g_clf_predict = g_clf.predict(count_vect.transform(new_x_test))

In [31]:
accuracy_score(new_y_test, g_clf_predict)

0.803030303030303

In [32]:
precision_score(new_y_test, g_clf_predict, average="weighted")

0.8409090909090909

In [33]:
recall_score(new_y_test, g_clf_predict, average="weighted")

0.803030303030303

In [34]:
f1_score(new_y_test, g_clf_predict, average="weighted")

0.7974025974025974

In [35]:
cm = confusion_matrix(new_y_test, gpt_r_predict)
print (cm)

[[53 46]
 [11 88]]
